This script will download the data required to reproduce the main figures.

> In order to download the files, you will need the base-url that is provided with the submission of the manuscript. 


In [ ]:
import getpass

base_url = getpass.getpass("Please enter the base URL")

In [ ]:
import os
import requests
import gzip
import shutil


def download_and_extract_file(file: str, base_url: str, force: bool = False):
    """Downloads and extracts a file from a specified base URL.

    Args:
        file (str): Name of the file to download.
        base_url (str): Base URL where the file is located.
        force (bool, optional): If True, re-downloads the file even if
                                it exists locally. Defaults to False.

    Raises:
        requests.exceptions.RequestException: If there is an error during download.
        (OSError, ValueError): If there is an error during extraction.
    """
    print(f"Downloading file {file}")

    # Ensure base_url ends with a slash
    if not base_url.endswith("/"):
        base_url += "/"
    full_url = f"{base_url}{file}"

    basename = os.path.basename(file)
    download_path = f"data/{file}"

    if basename.endswith(".gz"):
        download_path_extracted = download_path[:-3]
    else:
        download_path_extracted = download_path

    if os.path.exists(download_path_extracted):
        print(f"... File {file} is already downloaded")
        print(f"    to {download_path_extracted}")
        if force:
            os.remove(download_path_extracted)
            print("... Parameter force is set to true. Redownloading file.")
        else:
            print("    Set force=True to redownload the file.")
            return

    # Create the destination folder if it doesn't exist
    os.makedirs(os.path.dirname(download_path), exist_ok=True)

    try:
        response = requests.get(full_url, stream=True)
        response.raise_for_status()  # Raise an exception for error status codes

        with open(download_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):  # Chunked downloading
                f.write(chunk)

        if basename.endswith(".gz"):
            with gzip.open(download_path, "rb") as f_in:
                with open(download_path_extracted, "wb") as f_out:
                    shutil.copyfileobj(f_in, f_out)
            os.remove(download_path)  # Remove the extracted archive

    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")
    except (OSError, ValueError) as e:
        print(f"Error extracting file: {e}")

In [ ]:
files = [
    "xenium_output/day8_r2/morphology_mip.ome.tif",
    "xenium_output/day8_r2/experiment.xenium",
    "xenium_output/.DS_Store",
    "xenium_output/human_09_r2/morphology_mip.ome.tif",
    "xenium_output/human_09_r2/experiment.xenium",
    "transcripts/transcripts_figure_5c.csv.gz",
    ".DS_Store",
    "images/day8_r2_h_and_e_alignment_gan.npy.gz",
    "images/human_09_r2_IF_alignment.npy.gz",
    "images/human_09_r2_h_and_e_alignment_gan.npy.gz",
    "images/day8_r2_IF_alignment.npy.gz",
    "adata/.DS_Store",
    "adata/human.h5ad",
    "adata/human_09_r2_with_transcripts.h5ad",
    "adata/tgfb.h5ad",
    "adata/day8_r2_with_transcripts.h5ad",
    "adata/timecourse.h5ad",
    "IF/timecourse/day 120.txt",
    "IF/timecourse/day 060.txt",
    "IF/timecourse/day 006.txt",
    "IF/timecourse/day 007.txt",
    "IF/timecourse/day 005.txt",
]

In [ ]:
for f in files:
    download_and_extract_file(f, base_url, force=False)